In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading AI model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [43]:
import re

# Simple example
text = "The moon shines bright."
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# Generate 5 words, one at a time
current_text = text

newline_id = tokenizer.encode("\n", add_special_tokens=False)[0]

line_count = 1
words_in_line = 0

MIN_WORDS = 4
MAX_WORDS = 10
NEWLINE_BIAS = 2.0

#checks that token is valid
def is_valid_token(tok, newline_id, token_id):
    # allow newline
    if token_id == newline_id:
        return True
    # allow words
    if any(c.isalpha() for c in tok):
        return True
    # allow comma / period
    if tok.strip() in [",", "."]:
        return True
    return False

def get_lines(text):
    # treat comma, period, question, exclamation, and actual newlines as line breaks
    parts = re.split(r'(?:\n+|[,.!?]\s*)', text.strip())
    return [p.strip() for p in parts if p.strip()]

for step in range(40):

    print(f"--- Step {step + 1} ---")
    print(f"Current: '{current_text}'")

    # Encode current text
    input_ids = tokenizer.encode(current_text, return_tensors="pt")

    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # Get the predictions for the NEXT token
    next_token_logits = predictions[0, -1, :]
    for tid in tokenizer.all_special_ids:
      next_token_logits[tid] = -1e10

    temperature = 1.1
    next_token_logits = next_token_logits / temperature



    # Hard: don't allow newline too early (for lines 1-3)
    if line_count < 4 and words_in_line < MIN_WORDS:
        next_token_logits[newline_id] = -1e10

    # Hard: force newline if line too long (for lines 1-3)
    if line_count < 4 and words_in_line >= MAX_WORDS:
        next_token_logits[:] = -1e10
        next_token_logits[newline_id] = 0.0

    # Soft: encourage newline once we're past MIN_WORDS (for lines 1-3)
    if line_count < 4 and words_in_line >= MIN_WORDS:
        next_token_logits[newline_id] += NEWLINE_BIAS

    next_token_probs = torch.softmax(next_token_logits, dim=0)

    # Get top 5 predictions
    top_probs, top_indices = torch.topk(next_token_probs, 5)

    print("Top 5 next word predictions:")
    for i, (prob, idx) in enumerate(zip(top_probs, top_indices)):
        word = tokenizer.decode([idx])
        print(f"  {i+1}. '{word}' ({prob.item()*100:.1f}% confident)")

    # --- ADD: filter the top-k using your is_valid_token, then sample weighted ---
    filtered_probs = []
    filtered_indices = []

    for prob, idx in zip(top_probs, top_indices):
        tok = tokenizer.decode([idx])
        if is_valid_token(tok, newline_id, idx.item()):
            filtered_probs.append(prob)
            filtered_indices.append(idx)

    # fallback: if none pass, just take best token so we never hang
    if len(filtered_indices) == 0:
        next_token_id = top_indices[0]
    else:
        filtered_probs = torch.stack(filtered_probs)
        filtered_probs = filtered_probs / filtered_probs.sum()
        eps = 0.2  # 0.05–0.25
        filtered_probs = (1 - eps) * filtered_probs + eps * (torch.ones_like(filtered_probs) / len(filtered_probs))
        choice = torch.multinomial(filtered_probs, 1).item()
        next_token_id = filtered_indices[choice]

    next_word = tokenizer.decode([next_token_id])
    current_text += next_word
    if len(get_lines(current_text)) > 4:
      break

    # break line on sentence-ending punctuation (like a period)
    if "." in next_word and line_count < 4:
        current_text += "\n"
        line_count += 1
        words_in_line = 0

    # update word/line counts
    if next_token_id.item() == newline_id:
        line_count += 1
        words_in_line = 0
    else:
        # super-minimal "word count": count new words when token begins with a space
        if next_word.startswith(" "):
            words_in_line += 1

    print(f"✓ Chosen: '{next_word}'")
    print(f"New text: '{current_text}'\n")

lines = get_lines(current_text)[:4]
print("\nFinal generated text:")
for line in lines:
    print(line)


Starting text: 'The moon shines bright.'

Generating word by word...

--- Step 1 ---
Current: 'The moon shines bright.'
Top 5 next word predictions:
  1. ' The' (9.5% confident)
  2. ' It' (5.7% confident)
  3. ' But' (2.2% confident)
  4. ' And' (2.0% confident)
  5. ' A' (1.9% confident)
✓ Chosen: ' And'
New text: 'The moon shines bright. And'

--- Step 2 ---
Current: 'The moon shines bright. And'
Top 5 next word predictions:
  1. ' the' (8.6% confident)
  2. ' it' (6.2% confident)
  3. ' when' (3.5% confident)
  4. ' then' (3.1% confident)
  5. ' there' (2.7% confident)
✓ Chosen: ' when'
New text: 'The moon shines bright. And when'

--- Step 3 ---
Current: 'The moon shines bright. And when'
Top 5 next word predictions:
  1. ' the' (17.6% confident)
  2. ' it' (12.8% confident)
  3. ' you' (9.1% confident)
  4. ' we' (7.3% confident)
  5. ' I' (3.7% confident)
✓ Chosen: ' it'
New text: 'The moon shines bright. And when it'

--- Step 4 ---
Current: 'The moon shines bright. And when i